For this next tutorial we will dive  more into the parameter estimation performed by the fevd() function. 
As you might remember from the previous tutorials the GEV distribution has three variables: the location,
scale, and shape parameter. By default the fevd() function uses a 'maximum likelihood estimation' (MLE).
A MLE is an estimator, a function of your data that gives you an approximation of the parameters. In a 
MLE we compute the likelihood that a given set of parameters would return the observed data. the likelihood
is given by the earlier discussed probability density function that results from a set of parameters. 

As you might have noticed we thus made two very important assumptions in our previous tutorials... First
we assumed we were working with the GEV distribution, second we assumed the MLE method would give us the 
best parameter estimation. Here we will test these assumptions by first using different parameter estimation
methods to see if our parameter values signficiantly change. Next we show you how you can use different 
models.

But first lets open the precipitation data from Germany once more, and fit a GEV distribution to it using
the MLE method, you can do this by adding "method = MLE" to your fevd() function. Have a look at the parameter
values.

In [ ]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# #install dependencies - taken from <Yosmely Bermúdez> comments for Tutorial 6
# # We need this to install eigen which is needed for SDFC to install correctly
# !mamba install eigen numpy matplotlib seaborn pandas cartopy scipy texttable intake xarrayutils xmip cf_xarray intake-esm
# !pip install -v https://github.com/yrobink/SDFC/archive/master.zip#subdirectory=python
# !pip install https://github.com/njleach/mystatsfunctions/archive/master.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# import cartopy.crs as ccrs
from scipy import stats

In [ ]:
import extremes_functions as ef
from mystatsfunctions import OLSE,LMoments
import SDFC as sd

In [ ]:
def estimate_return_level(quantile,model):
    loc, scale, shape = model.coef_
    level = loc - scale / shape * (1 - (-np.log(quantile))**(-shape))
    # level = stats.genextreme.ppf(quantile,-shape,loc=loc,scale=scale)
    return level

In [ ]:
import os, pooch

fname = 'precipitationGermany_1920-2022.csv'
if not os.path.exists(fname):
    url = "https://osf.io/xs7h6/download"
    fname = pooch.retrieve(url, known_hash=None)

data = pd.read_csv(fname, index_col=0).set_index('years')

data.columns=['precipitation']
precipitation = data.precipitation

In [ ]:
fit, model = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True)

There are two other parameter estimation methods that we will explore and compare with the MLE: the
L-moments method, and a Baseyian method. The L-moments describe the shape of a probability distribution,
just like regular moments, and are a linear combination of the order statistics (L stands for linear).
The GEV-distribution parameters can be computed through a set of equations based from these L-moments.

The Bayesian technique on the other hand, [explain]

Estimate the GEV parameters based on these two method by changing the 'method' to either "Lmoments" and 
"Bayesian". After that compute the 100-year flood for all three parameter sets (MLE, Lmoments and Bayesian)

In [ ]:
fit_moments, model_moments = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True,method='moments')

In [ ]:
fit_moments

In [ ]:
model_moments

In [ ]:
prior = stats.multivariate_normal(mean= model.coef_, cov = np.cov(model.coefs_bootstrap.T), allow_singular=True)

In [ ]:
fit_bayes, model_bayes = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True,method='bayesian',prior=prior,mcmc_init=model.coef_)

In [ ]:
fit_bayes

In [ ]:
model_bayes

In [ ]:
period = 100
quantile = 1-1/period

print('MLE: %.2f' % estimate_return_level(quantile,model))
print('Moments: %.2f' % estimate_return_level(quantile,model_moments))
print('Bayes: %.2f' % estimate_return_level(quantile,model_bayes))

In [ ]:
period = 50
quantile = 1-1/period

print('MLE: %.2f' % estimate_return_level(quantile,model))
print('Moments: %.2f' % estimate_return_level(quantile,model_moments))
print('Bayes: %.2f' % estimate_return_level(quantile,model_bayes))

In [ ]:
period = 500
quantile = 1-1/period

print('MLE: %.2f' % estimate_return_level(quantile,model))
print('Moments: %.2f' % estimate_return_level(quantile,model_moments))
print('Bayes: %.2f' % estimate_return_level(quantile,model_bayes))

### Fit Moments using a different method

In [ ]:
gev = LMoments.gev()
gev.fit(precipitation.values)
gev.X, gev.a, gev.k